First we need to include the source files for the BMP functions.

In [ ]:
include("../src/BMP.jl")

# Constructing BMPs

Two constructors are provided for a BMP. One of these creates a BMP for a function that selects a single variable (i.e. $f(x_1, \dotsc, x_n) = x_i$ for some $i \in \{1,2,\dotsc,n\}$) and the other creates a BMP for the constant functions ($f(x_1,\dotsc,x_n) = 1$ or $f(x_1,\dotsc,x_n) = 0$). More complicated functions can be sythnesized from these building blocks.

In [ ]:
x1 = BMP_bitline(1, 3) # BMP for variable 1 of three variables
x2 = BMP_bitline(2, 3)
x3 = BMP_bitline(3, 3)
cnst0 = BMP(0, 3)
cnst1 = BMP(1, 3)

The most important BMP operation is APPLY. This method takes the BMPs for a number of functions $g_1(\vec{x}), g_2(\vec{x}), \dotsc, g_k(\vec{x})$, and produces the BMP for the composite function
$$
f\left( g_1(\vec{x}), g_2(\vec{x}), \dotsc, g_k(\vec{x}) \right).
$$
The APPLY operation needs as inputs the BMPs for $g_i(\vec{x})$ and the truth table of the function $f$. The truth table is specified as a simple array of length $2^k$. The (0-based) indices of this array correspond to the values of the $k$ input values, with the most significant bit being the value of the first input variable. (So that the truth table array for $f(g_1, g_2) = \overline{g_1} g_2$ is $[0,1,0,0]$, as a simple example.)

Now, let's build the BMP for the function
$$
f(x_1, x_2, x_3) = x_1 x_2 + \overline{x_3}.
$$
We can do this step by step using the simple gates AND, OR, and NOT. In addition, NOT can be replaced by XOR'ing with constant 1. We first define the truth tables of these gates for convenience.

In [ ]:
gAND = [0, 0, 0, 1]
gOR = [0, 1, 1, 1]
gXOR = [0, 1, 1, 0]

The BMP is then constructed using the following piece of code.

In [ ]:
t1 = BMP_apply(x1, x2, gAND) # The first term
t2 = BMP_apply(x3, cnst1, gXOR) # The second term
bmp = BMP_apply(t1, t2, gOR)

To verify that the correct BMP has been generated, we can check its output for all possible inputs. `eval` function evaluates the BMP for a given set of input values. These input values are given as an array. The size of the array along the first dimension must be the number of input bits, the rest of the dimensions can be anything. For $n$ inputs and $m$ outputs, if the input array has dimensions $(n, d_1, d_2, \dotsc)$ the output array will have dimensions $(m, d_1, d_2, \dotsc)$.

In [ ]:
all_inputs = [j >> (3-i) & 1 for i=1:3, j=0:7] # 3x8 array containing inputs
all_outputs = eval(bmp, all_inputs) # The output is 1x8
f_out = reshape([(i1 & i2) | (~i3 & 1) for i3=0:1, i2=0:1, i1=0:1], (1,8)) # Output computed directly
for j in axes(all_inputs,2)
    for i in axes(all_inputs,1)
        print(" $(all_inputs[i,j])")
    end
    print(" | $(all_outputs[1,j])\n")
end
print("Input tests: ")
print(all(f_out .== all_outputs) ? "PASSED\n" : "FAILED\n")

Alternatively, the BMP can be built in a single step using the truth table of $f$.

In [ ]:
bmp1 = BMP_apply([x1, x2, x3], reshape(f_out, 8))
@show all(eval(bmp, all_inputs) .== eval(bmp1, all_inputs))

# Evaluation

The matrices that comprise the BMP are row-switching, which means that they have a single non-zero entry equal to one on each row. These matrices can be represented by one-dimensional vectors whose entries indicate the non-zero column indices of rows, as in
$$A = \begin{bmatrix} 0 & 1 & 0 \\ 1 & 0 & 0 \\ 0 & 0 & 1 \end{bmatrix}
\to
a = [2, 1, 3].
$$

Such matrices can be efficiently multiplied as well. If $A$ and $B$ are both such matrices represented by arrays $a$ and $b$, the array representing $C = AB$ is simply $c[i] = b[a[i]]$.

Given a BMP of matrices $M^{(x_1)}_1, M^{(x_2)}_2, \dotsc, M^{(x_n)}_n$, a terminal vector $R$, and the input $\vec{v} = (v_1, v_2, \dotsc, v_n)$, the output is simply the product
$$
M^{(v_1)}_1 \cdot M^{(v_2)}_2 \cdot \dotsc \cdot M^{(x_{n-1})}_{n-1} \cdot M^{(x_n)}_n \cdot R.
$$
For general matrices, the optimal order to perform these multiplications can be found using dynamic programming. This is unnecessary for BMPs because with the vector representation of row-switching matrices the cost of a single multiplication only depends on the number of rows of the left matrix. Furthermore, in a generic BMP, the leftmost matrix will usually be the smallest one. Therefore we can simply start with the leftmost matrix and go through the chain left-to-right. We can simply initiate an array $t \gets m^{(v_1)}_1$ and repeatedly update this as $t[i] \gets m^{(v_k)}_k[t[i]]$ for $k=2,\dotsc,n$. The size of the array $t$ doesn't need to change, so only one memory allocation is necessary.

This approach is very similar to a graph traversal. We start at a particular row of the first matrix and jump to a row on the second matrix following the non-zero entry of the initial row. It is implemented in the code with the following functions.

In [ ]:
function eval(bmp::BareBMP, x::BitArray, R::Vector{<:Integer}, order::Vector{<:Integer})::BitArray
    n = size(bmp, 1)
    m = length(bmp[1,1].rows)
    n_samps = div(length(x), size(x, 1))
    x_ = reshape(x, (n, n_samps))
    result = BitArray(undef, (m, n_samps))
    mat = fill(RSMInt(0), m)
    for j=1:n_samps
        mat .= bmp[1, x_[order[1],j] + 1].rows
        for i=2:n
            RSM_mult_inplace(mat, bmp[i, x_[order[i], j] + 1])
        end
        for k=1:m
            result[k,j] = R[mat[k]]
        end
    end
    shape = [i for i in size(x)]
    shape[1] = m
    return reshape(result, Tuple(shape))
end

function eval(bmp::BMP, x::BitArray)::BitArray
    return eval(bmp.M, x, bmp.R, bmp.order)
end

function eval(bmp::BMP, x::Array{<:Integer})
    f = eval(bmp, x .!= 0)
    return Array{eltype(x)}(f)
end

The important lines here are 7 through 16. The storage for outputs is allocated on line 7. The loop starting in the following line iterates over all sets of inputs given for the function. (The user can evaluate multiple values with a single call.) On line 9 the output vector is filled with the values of the first matrix array. The first inner loop is all the matrix multiplications performed in place, i.e. the results are written directly into `mat`. The second inner loop is the multiplication with the terminal vector $R$ whose result is written into the full result array.

Note that the input bits are indexed through an array called `order`. This is because the variable ordering of BMPs can be changed, as explained below.

# Multiple outputs

Often we want to work with BMPs that have multiple outputs. We can create such a BMP from a collection of single output BMPs using the `BMP_join` function. As an example, we can build an adder that has $2n$ inputs (two $n$-bit numbers) and $n+1$ outputs.

In [ ]:
function adder_bits(n::Integer)
    carry = BMP(0, 2*n)
    outputs = Vector{BMP}(undef, n+1)
    for i=n:-1:1
        xi = BMP_bitline(i, 2*n)
        yi = BMP_bitline(i+n, 2*n)
        outputs[i+1] = BMP_apply([xi, yi, carry], [0, 1, 1, 0, 1, 0, 0, 1])
        carry = BMP_apply([xi, yi, carry], [0, 0, 0, 1, 0, 1, 1, 1])
    end
    outputs[1] = carry
    return outputs
end

outputs8 = adder_bits(8)
adder8 = BMP_join(outputs8)

# Test the adder
adder_inputs = [(i << 8 | j) >> k & 1 for k=15:-1:0, j=0:255, i=0:255]
adder_outputs = [(i+j) >> k & 1 for k=8:-1:0, j=0:255, i=0:255]
@show all(eval(adder8, adder_inputs) .== adder_outputs)

# BMP size and cleaning

The BMP is essentially a collection of 'row-switching' matrices, i.e. matrices with a single non-zero column on each row. Such matrices can be stored as one-dimensional arrays of numbers that indicate the non-zero column of each row. The space required for a BMP then is proportional to the sum of row numbers of the matrices, plus the length of the terminal vector. The functions `BMP_volume` computes this quantity.

In [ ]:
@show BMP_volume(adder8)

The sizes of individual matrices can be accessed using `BMP_dims` as shown below.

In [ ]:
@show BMP_dims(adder8)
@show BMP_dims(adder8, 1)
@show BMP_dims(adder8, 9)

The BMP for a function is not unique, since it may contain redundancies. In general we want to get rid of these and obtain the smallest possible BMP. This is what the CLEAN operation does. This operation can be performed left-to-right in the matrix chain or right-to-left, the two versions are not interchangeable but achieve different results. Depending on how a BMP is constructed, a right-to-left sweep may suffice, but there are cases where both are necessary.

The APPLY operation as implemented by `BMP_apply` in fact produces a redundant BMP first, and uses the CLEAN operation to generate the minimized BMP. `BMP_apply_noclean` skips this step. We can see the differences in size for the function we constructed in the first section.

In [ ]:
@show BMP_volume(bmp1)
bmp2 = BMP_apply_noclean([x1, x2, x3], reshape(f_out, 8))
@show BMP_volume(bmp2)
@show all(eval(bmp1, all_inputs) .== eval(bmp2, all_inputs))

We can clean this both ways to see how the BMP size decreases.

In [ ]:
# Left-to-right cleaning
bmp2_lr = BMP_clean1_lr(bmp2)
@show BMP_volume(bmp2_lr)
@show all(eval(bmp2_lr, all_inputs) .== eval(bmp2, all_inputs))

# Right-to-left cleaning
bmp2_rl = BMP_clean1_rl(bmp2)
@show BMP_volume(bmp2_rl)
@show all(eval(bmp2_rl, all_inputs) .== eval(bmp2, all_inputs))

# Left-to-right, followed by right-to-left
bmp2_ = BMP_clean1(bmp2)
@show BMP_volume(bmp2_)
@show all(eval(bmp2_, all_inputs) .== eval(bmp2, all_inputs))

Note that while it looks like the right-to-left cleaning was sufficient for this specific function, it is in general necessary to do two sweeps when using `BMP_apply`. There is a different implementation of APPLY that's called `BMP_minapply`, for which the right-to-left sweep is always enough to reach the minimum BMP. This is because the redundancies removed by the left-to-right sweep do not show up at all with this method. As a result, it is faster in a lot of cases. However, because it uses a more complicated algorithm, there are circumstances where `BMP_apply` is more efficient.

# Variable ordering

By default, the variables in the BMP are ordered $1$ through $n$ from left-to-right. We can see this by accessing the `order` field of the BMP. (The output below is in hexadecimal since this is default behavior in Julia for unsigned integers.)

In [ ]:
@show bmp1.order
@show adder8.order

We can construct a BMP with a particular ordering from the beginning, or change the ordering after the construction. The former is done by using the appropriate constructors.

In [ ]:
x1_ = BMP_bitline(1, [3,2,1]) # BMP for variable 1 with variable ordering [3,2,1]
x2_ = BMP_bitline(2, [3,2,1])
x3_ = BMP_bitline(3, [3,2,1])
cnst1_ = BMP(1, [3,2,1])
bmp1_rev = BMP_apply([x1_, x2_, x3_], reshape(f_out, 8))
@show bmp1_rev.order
@show all(eval(bmp1_rev, all_inputs) .== eval(bmp1, all_inputs))

Note that the internal variable ordering does not affect the ordering used for the inputs when calling `eval`. One therefore needs to distinguish the variable labels and the positions on the BMP. The first variable from the left is not necessarily the variable with label $1$. `eval`, `BMP_bitline`, `BMP` functions all refer to variable labels and not positions.

Also note that APPLY can only be used on BMPs that have the same variable orderings. This has to be ensured by the user as the functions `BMP_apply` and `BMP_minapply` do not perform this check.

In order to change the variable ordering after construction we can use SWAP. This switches the order of the variable at a particular position and the variable to the right of it. Unlike the functions above, SWAP refers to the position on the internal variable ordering and not to the variable label.

In [ ]:
BMP_swap!(bmp1_rev, 1) # Switch the variables at positions 1 and 2
@show bmp1_rev.order
@show all(eval(bmp1_rev, all_inputs) .== eval(bmp1, all_inputs))

Importantly, `BMP_swap!` modifies the input BMP, instead of returning a new one.

The variable ordering can have a dramatic effect on the BMP volume. This cannot really be seen for the simple function we have above but is much more clear for the adder, as seen below.

In [ ]:
function adder_bits_ordered(n::Integer)
    carry = BMP(0, 2*n)
    outputs = Vector{BMP}(undef, n+1)
    order = reshape([i + v * n for v=0:1, i=1:n], 2*n)
    @show order
    for i=n:-1:1
        xi = BMP_bitline(i, order)
        yi = BMP_bitline(i+n, order)
        outputs[i+1] = BMP_apply([xi, yi, carry], [0, 1, 1, 0, 1, 0, 0, 1])
        carry = BMP_apply([xi, yi, carry], [0, 0, 0, 1, 0, 1, 1, 1])
    end
    outputs[1] = carry
    return outputs
end

outputs8_ord = adder_bits_ordered(8)
adder8_ord = BMP_join(outputs8_ord)
@show BMP_volume(adder8_ord)
@show BMP_volume(adder8)
@show all(eval(adder8_ord, adder_inputs) .== adder_outputs)

The default variable ordering leads to a BMP that is larger by more than an order of magnitude. Given this situation, it makes sense to search for the optimal variable ordering for a given function. This can be done before constructing a BMP for certain classes of functions, assuming their form known beforehand. We are more concerned with the case where the function is already represented by a BMP. In such cases, there exists an exact method of exponential complexity, which is a version of best-first search with some additional optimizations adapted to this problem. It is implemented by `exact_minimize` method in the `BMP_ordering.jl` file.

In [ ]:
include("../src/BMP_ordering.jl")

# Initial variable ordering and volume
@show Int.(adder8.order)
@show BMP_volume(adder8)

# Optimized variable ordering and volume
@time exact_minimize!(adder8)
@show Int.(adder8.order)
@show BMP_volume(adder8)
@show all(eval(adder8, adder_inputs) .== adder_outputs)

Exact minimization can be quite slow. Usually it's a better idea to try a heuristic algorithm called 'sifting' to see if it produces a reasonable small BMP.

In [ ]:
adder8 = BMP_join(adder_bits(8))
@show Int.(adder8.order)
@show BMP_volume(adder8)

@time sift!(adder8)
@show Int.(adder8.order)
@show BMP_volume(adder8)
@show all(eval(adder8, adder_inputs) .== adder_outputs)

In this example the sifting algorithm finds the optimal variable ordering in a fraction of time, but it usually won't perform so well. While the runtime should remain tiny compared to the exact method, the variable ordering it comes up with will be a local minimum of BMP volume a few times larger than that of the optimal ordering. In fact, this will be the case for the adder as well if it's initialized with the opposite variable ordering.

In [ ]:
function adder_bits_rev(n::Integer)
    carry = BMP(0, 2*n)
    outputs = Vector{BMP}(undef, n+1)
    order = reverse(collect(1:2*n))
    @show order
    for i=n:-1:1
        xi = BMP_bitline(i, order)
        yi = BMP_bitline(i+n, order)
        outputs[i+1] = BMP_apply([xi, yi, carry], [0, 1, 1, 0, 1, 0, 0, 1])
        carry = BMP_apply([xi, yi, carry], [0, 0, 0, 1, 0, 1, 1, 1])
    end
    outputs[1] = carry
    return outputs
end

outputs8_rev = adder_bits_rev(8)
adder8_rev = BMP_join(outputs8_rev)
@show BMP_volume(adder8_rev)
@time sift!(adder8_rev)
@show BMP_volume(adder8_rev)
@show all(eval(adder8_rev, adder_inputs) .== adder_outputs)